In [8]:
import matplotlib.pyplot as plt

In [9]:
from mog_experiment import *

In [10]:
X_DIM = 2
NUM_DISTRIBUTIONS = 2

MEANS = np.zeros((X_DIM, X_DIM), dtype=np.float32)
CENTERS = np.sqrt(2)  # center of Gaussian
for i in range(NUM_DISTRIBUTIONS):
    MEANS[i::NUM_DISTRIBUTIONS, i] = CENTERS

params = {                          # default parameter values
    'x_dim': X_DIM,
    'num_distributions': NUM_DISTRIBUTIONS,
    'means': MEANS,
    'sigma': 0.05,
    'small_pi': 2E-16,
    'scale': 0.1,
    'num_samples': 200,
    'lr_init': 1e-3,
    'lr_decay_steps': 1000,
    'eps': 0.3,
    'temp_init': 20,
    'annealing_steps': 200,
    'annealing_rate': 0.98,
    'train_trajectory_length': 10,
    'test_trajectory_length': 2000,
    'num_training_steps': 25000,
    'tunneling_rate_steps': 1000,
    'save_steps': 2500,
    'logging_steps': 100
}

In [11]:
tf.reset_default_graph()

In [12]:
model = GaussianMixtureModel(params)

Creating directory for new run: ../log_mog_tf/run56/


In [ ]:
#config = tf.ConfigProto(log_device_placement=True)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
model.build_graph()
model.train(params['num_training_steps'], config=config, plot=False)

params file written to: ../log_mog_tf/run56/run_info/parameters.txt
Step: 0/25000, Loss: -192.8, accept rate: 0.92, LR: 0.001, temp: 20, step size: 0.3

Step: 100/25000, Loss: -201.4, accept rate: 0.96, LR: 0.001, temp: 19.6, step size: 0.308

Step: 200/25000, Loss: -222.7, accept rate: 0.95, LR: 0.001, temp: 19.6, step size: 0.309

Step: 300/25000, Loss: -218.7, accept rate: 0.94, LR: 0.001, temp: 19.208, step size: 0.308

Step: 400/25000, Loss: -208.2, accept rate: 0.91, LR: 0.001, temp: 19.208, step size: 0.314

Step: 500/25000, Loss: -214.1, accept rate: 0.91, LR: 0.001, temp: 18.824, step size: 0.312

Step: 600/25000, Loss: -252, accept rate: 0.91, LR: 0.001, temp: 18.824, step size: 0.31

Step: 700/25000, Loss: -219.4, accept rate: 0.95, LR: 0.001, temp: 18.447, step size: 0.309

Step: 800/25000, Loss: -237, accept rate: 0.95, LR: 0.001, temp: 18.447, step size: 0.309

Step: 900/25000, Loss: -241.8, accept rate: 0.96, LR: 0.001, temp: 18.078, step size: 0.308


	Step: 1001, Tunne

In [ ]:
config = tf.ConfigProto(log_device_placement=True)
config.gpu_options.allow_growth = True
model.build_graph()
model.train(params['num_training_steps'], config=config, plot=False)

params file written to: ../log_mog_tf/run54/run_info/parameters.txt
Step: 0/25000, Loss: -195.8, accept rate: 0.92, LR: 0.001, temp: 20, step size: 0.3

Step: 100/25000, Loss: -212.2, accept rate: 0.96, LR: 0.001, temp: 19.6, step size: 0.311

Step: 200/25000, Loss: -231.3, accept rate: 0.94, LR: 0.001, temp: 19.6, step size: 0.314

Step: 300/25000, Loss: -212.2, accept rate: 0.92, LR: 0.001, temp: 19.208, step size: 0.311

Step: 400/25000, Loss: -209.4, accept rate: 0.93, LR: 0.001, temp: 19.208, step size: 0.31

Step: 500/25000, Loss: -220.7, accept rate: 0.91, LR: 0.001, temp: 18.824, step size: 0.306

Step: 600/25000, Loss: -213.1, accept rate: 0.89, LR: 0.001, temp: 18.824, step size: 0.303

Step: 700/25000, Loss: -233.6, accept rate: 0.93, LR: 0.001, temp: 18.447, step size: 0.302

Step: 800/25000, Loss: -233.4, accept rate: 0.95, LR: 0.001, temp: 18.447, step size: 0.302

Step: 900/25000, Loss: -261.6, accept rate: 0.96, LR: 0.001, temp: 18.078, step size: 0.3


	Step: 1001, Tun